In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np 
sns.set_style('whitegrid')
%matplotlib inline
import numpy as np
from sklearn.preprocessing import MinMaxScaler


from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
import xgboost as xgb 
from sklearn import ensemble 
from folium.plugins import HeatMap
plt.style.use('ggplot')


from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, matthews_corrcoef
import folium
from folium.plugins import MarkerCluster

ModuleNotFoundError: No module named 'seaborn'

: 

In [ ]:
df = pd.read_csv("archive(3)/US_Accidents_March23.csv")

In [ ]:
df.head(10)

In [ ]:
df.columns

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.Source.unique()


In [ ]:
states = df.State.unique()


In [ ]:
count_by_state=[]
for i in df.State.unique():
    count_by_state.append(df[df['State']==i].count()['ID'])

fig,ax = plt.subplots(figsize=(16,10))
sns.barplot(count_by_state)

In [ ]:
sns.jointplot(x=df.Start_Lat.values,y=df.Start_Lng.values,height=10)
plt.ylabel('Start_Lat', fontsize=12)
plt.xlabel('Start_Lng', fontsize=12)
plt.show()

In [ ]:
sns.jointplot(x=df.End_Lat.values,y=df.End_Lng.values,height=10)
plt.ylabel('End_Lat', fontsize=12)
plt.xlabel('End_Lng', fontsize=12)
plt.show()

In [ ]:
fig, ax=plt.subplots(figsize=(16,7))
df['Weather_Condition'].value_counts().sort_values(ascending=False).head(5).plot.bar(width=0.5,edgecolor='k',align='center',linewidth=2)
plt.xlabel('Weather_Condition',fontsize=20)
plt.ylabel('Number of Accidents',fontsize=20)
ax.tick_params(labelsize=20)
plt.title('5 Top Weather Condition for accidents',fontsize=25)
plt.grid()
plt.ioff()

In [ ]:
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df

In [ ]:
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['columns_name','missing_count']
missing_df['missing_ratio'] = missing_df['missing_count'] /df.shape[0]
missing_df.loc[missing_df['missing_ratio']>0.777]

In [ ]:
missin = missing_df.loc[missing_df['missing_count']>250000]
removelist = missin['columns_name'].tolist()
removelist

In [ ]:
df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')
df['End_Time'] = pd.to_datetime(df['End_Time'], errors='coerce')

# Extract year, month, day, hour and weekday
df['Year']=df['Start_Time'].dt.year
df['Month']=df['Start_Time'].dt.strftime('%b')
df['Day']=df['Start_Time'].dt.day
df['Hour']=df['Start_Time'].dt.hour
df['Weekday']=df['Start_Time'].dt.strftime('%a')

# Extract the amount of time in the unit of minutes for each accident, round to the nearest integer
td='Time_Duration(min)'
df[td]=round((df['End_Time']-df['Start_Time'])/np.timedelta64(1,'m'))

In [ ]:
neg_outliers=df[td]<=0

# Set outliers to NAN
df[neg_outliers] = np.nan

# Drop rows with negative td
df.dropna(subset=[td],axis=0,inplace=True)

In [ ]:
feature_list=df.columns
df_copy = df[feature_list].copy()
df_copy.info()

In [ ]:
x_cols = [col for col in df_copy.columns if col not in ['Severity'] if df_copy[col].dtype=='float64']

labels = []
values = []
for col in x_cols:
    labels.append(col)
    values.append(np.corrcoef(df[col].values, df.Severity.values)[0,1])
corr_df = pd.DataFrame({'col_labels':labels, 'corr_values':values})
corr_df = corr_df.sort_values(by='corr_values')

ind = np.arange(len(labels))
width = 0.9
fig, ax = plt.subplots(figsize=(12,40))
rects = ax.barh(ind, np.array(corr_df.corr_values.values), color='y')
ax.set_yticks(ind)
ax.set_yticklabels(corr_df.col_labels.values, rotation='horizontal')
ax.set_xlabel("Correlation coefficient")
ax.set_title("Correlation coefficient of the variables")
plt.show()

In [ ]:
corr_zero_columns = ['Turning_Loop','Visibility(mi)','Pressure(in)','Humidity(%)','Temperature(F)']
for col in corr_zero_columns:
    print(col,len(df_copy[col].unique()))

In [ ]:
corr_df_sel = corr_df.loc[(corr_df['corr_values']>0.05) | (corr_df['corr_values'] < -0.05)]
corr_df_sel

In [ ]:
corr_df_ = corr_df_sel.col_labels.tolist()

tem_df = df[corr_df_]

corrmat = tem_df.corr(method='spearman')
fig,ax= plt.subplots(figsize=(8,8))

sns.heatmap(corrmat,vmax=1,square = True)
plt.title('corr map',fontsize=15)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,10)) 
fig_dims = (3, 2)


plt.subplot2grid(fig_dims, (0, 0))
df['Amenity'].value_counts().plot(kind='bar', 
                                     title='Amenity')
plt.subplot2grid(fig_dims, (0, 1))
df['Crossing'].value_counts().plot(kind='bar', 
                                     title='Crossing')
plt.subplot2grid(fig_dims, (1, 0))
df['Junction'].value_counts().plot(kind='bar', 
                                     title='Junction')
plt.subplot2grid(fig_dims, (1, 1))
df['Junction'].value_counts().plot(kind='bar', 
                                     title='Junction')

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x="Severity", y="Wind_Chill(F)", data=df)
plt.ylabel('Wind_Chill(F)', fontsize=12)
plt.xlabel('Severity', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.violinplot(x='Severity', y='Amenity', data=df)
plt.xlabel('Severity', fontsize=12)
plt.ylabel('Amenity', fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.violinplot(x='Severity', y='Wind_Chill(F)', data=df)
plt.xlabel('Severity', fontsize=12)
plt.ylabel('Wind_Chill(F)', fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.violinplot(x='Severity', y='Crossing', data=df)
plt.xlabel('Severity', fontsize=12)
plt.ylabel('Crossing', fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.violinplot(x='Severity', y='Junction', data= df)
plt.xlabel('Severity', fontsize=12)
plt.ylabel('Junction', fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.violinplot(x='Severity', y='Traffic_Signal', data=df)
plt.xlabel('Severity', fontsize=12)
plt.ylabel('Traffic_Signal', fontsize=12)
plt.show()

In [ ]:
df.dropna(subset=df.columns[df.isnull().mean()!=0], how='any', axis=0, inplace=True)
df.shape

In [ ]:
train_y = df['Severity'].values
x_cols = [col for col in df.columns if col not in ['Severity'] if df[col].dtype=='float64']
train_col= df[x_cols]

fearture_name = train_col.columns.values 

from sklearn import ensemble 

model = ensemble.ExtraTreesRegressor(n_estimators=25, max_depth=30, max_features=0.3, n_jobs=-1, random_state=0)
model.fit(train_col,train_y)

#plot imp 
importance = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_],axis=0)
indices = np.argsort(importance)[::-1][:20]

plt.figure(figsize=(12,12))
plt.title("Feature importances")
plt.bar(range(len(indices)), importance[indices], color="r", yerr=std[indices], align="center")
plt.xticks(range(len(indices)), fearture_name[indices], rotation='vertical')
plt.xlim([-1, len(indices)])
plt.show()

In [ ]:
import xgboost as xgb 

xgb_prames = {
    'eta': 0.05,
    'max_depth': 8,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'silent': 1,
    'seed' : 0
}

dtrain = xgb.DMatrix(train_col,train_y,feature_names=train_col.columns.values)

model = xgb.train(dict(xgb_prames, silent=0), dtrain, num_boost_round=50)


fig, ax = plt.subplots(figsize=(12,18))
xgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
plt.show()

## Geospartial analysis

In [ ]:
def create_map(df_loc, latitude, longitude, zoom, tiles='OpenStreetMap'):
    """
    Generate a Folium Map with clustered markers of accident locations.
    """
    world_map = folium.Map(location=[latitude, longitude], zoom_start=zoom, tiles=tiles)
    marker_cluster = MarkerCluster().add_to(world_map)

    # Iterate over the DataFrame rows and add each marker to the cluster
    for idx, row in df_loc.iterrows():
        folium.Marker(
            location=[row['Start_Lat'], row['Start_Lng']],
            # You can add more attributes to your marker here, such as a popup
            popup=f"Lat, Lng: {row['Start_Lat']}, {row['Start_Lng']}"
        ).add_to(marker_cluster)

    return world_map

In [ ]:
us_cities_coords = {
    "New York": {"lat": 40.7128, "lon": -74.0060},
    "Los Angeles": {"lat": 34.0522, "lon": -118.2437},
    "Chicago": {"lat": 41.8781, "lon": -87.6298},
    "Houston": {"lat": 29.7604, "lon": -95.3698},
    "Phoenix": {"lat": 33.4484, "lon": -112.0740},
    "Philadelphia": {"lat": 39.9526, "lon": -75.1652},
    "San Antonio": {"lat": 29.4241, "lon": -98.4936},
    "San Diego": {"lat": 32.7157, "lon": -117.1611},
    "Dallas": {"lat": 32.7767, "lon": -96.7970},
    "San Jose": {"lat": 37.3382, "lon": -121.8863},
    "Austin": {"lat": 30.2672, "lon": -97.7431},
    "Jacksonville": {"lat": 30.3322, "lon": -81.6557},
    "Fort Worth": {"lat": 32.7555, "lon": -97.3308},
    "Columbus": {"lat": 39.9612, "lon": -82.9988},
    "San Francisco": {"lat": 37.7749, "lon": -122.4194},
    "Charlotte": {"lat": 35.2271, "lon": -80.8431},
    "Indianapolis": {"lat": 39.7684, "lon": -86.1581},
    "Seattle": {"lat": 47.6062, "lon": -122.3321},
    "Denver": {"lat": 39.7392, "lon": -104.9903},
    "Washington": {"lat": 38.9072, "lon": -77.0369}
}

In [ ]:
df_loc = df.loc[(~df.Start_Lat.isna()) & (~df.Start_Lng.isna())]

### Mapping US car accidents

In [ ]:


#Create and display a Folium map with clustered markers for accident locations 
# centered on the US geographical center.
map_us = create_map(df_loc, 39.50, -98.35, 4)
map_us




### Mapping Using Car Accidents - Using Cartodbdark Matter Tiles

In [ ]:


map_us = create_map(df_loc, 39.50, -98.35, 4, tiles="Cartodbdark_matter")
map_us



### Mapping New York Car Accidents - Using Cartodbpositron Tiles

In [ ]:


map_ny = create_map(
    df_loc, us_cities_coords['New York']['lat'],
    us_cities_coords['New York']['lon'],
    10, tiles="cartodbpositron")
map_ny



### 
Mapping Los Angeles Car Accidents - Cartodbdark Matter Tiles

In [ ]:


map_ls = create_map(df_loc, 
                    us_cities_coords['Los Angeles']['lat'],
                    us_cities_coords['Los Angeles']['lon'], 
                    10,
                    tiles="Cartodbdark_matter"
                   )
map_ls



### Mapping Los Angeles Car Accidents- Using Folium Heatmap

In [ ]:
from folium.plugins import HeatMap

def create_heatmap(df_loc, latitude, longitude, zoom, tiles='OpenStreetMap'):
    """
    Generate a Folium Map with a heatmap of accident locations.
    """
    # Create a list of coordinates from the dataframe columns 'Start_Lat' and 'Start_Lng'
    heat_data = [[row['Start_Lat'], row['Start_Lng']] for index, row in df_loc.iterrows()]

    # Create a map centered around the specified coordinates
    world_map = folium.Map(location=[latitude, longitude], zoom_start=zoom, tiles=tiles)

    # Add the heatmap layer to the map
    HeatMap(heat_data).add_to(world_map)

    return world_map


# Create a heatmap for the entire US using the create_heatmap function
map_us_heatmap = create_heatmap(df_loc.sample(3000), us_cities_coords['Los Angeles']['lat'],
                                us_cities_coords['Los Angeles']['lon']
                                ,  10)

map_us_heatmap

### Simple CircleMarker Accident Map of US Car Accidents

In [ ]:
def create_accident_map(df_loc):
    """Generate a map with CircleMarkers for accident locations."""
    m = folium.Map(location=[39.8283, -98.5795], width="100%", height="100%", zoom_start=4)
    
    for index, row in df_loc.iterrows():
        lat = row['Start_Lat']
        lon = row['Start_Lng']
        
        if not pd.isnull(lat) and not pd.isnull(lon):
            folium.CircleMarker(
                location=[lat, lon],
                radius=1,
                color='blue',
                fill=True,
                fill_color='blue',
                fill_opacity=0.6
            ).add_to(m)

    return m

create_accident_map(df_loc)

### Accident Severity Map with Colored Markers

In [ ]:
def create_accident_map_with_colors(df_loc):
    """Generate a map with CircleMarkers for accident locations, colored by severity."""
    m = folium.Map(location=[39.8283, -98.5795], width="100%", height="100%", zoom_start=4)
    
    # color scheme for different severity levels 
    severity_colors = {
        1: 'purple',
        2: 'green',
        3: 'orange',
        4: 'red',
    }
    
    for index, row in df_loc.iterrows():
        lat = row['Start_Lat']
        lon = row['Start_Lng']
        severity = row['Severity']
        
        if not pd.isnull(lat) and not pd.isnull(lon) and severity in severity_colors:
            # Get color based on severity
            color = severity_colors[severity]
            
            folium.CircleMarker(
                location=[lat, lon],
                radius=1,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.6
            ).add_to(m)
    return m

create_accident_map_with_colors(df_loc)
Make this Notebook Trusted to 